In [ ]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import joblib
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, Dropout, Conv1D, LayerNormalization,
    MultiHeadAttention, GlobalAveragePooling1D, Add
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from keras_tuner import HyperModel
from keras_tuner.tuners import Hyperband

In [ ]:
param_df = pd.read_excel('images_index.xlsx') 
param_df.set_index('name', inplace=True)


curve_folder = './curves'  
all_curves = {}
for file in glob(os.path.join(curve_folder, '*.xlsx')):
    df = pd.read_excel(file)
    for col in df.columns[1:]:  
        if df[col].isnull().any():
            continue
        all_curves[col] = df[col].values[:200]  

X, Y = [], []
for name in param_df.index:
    if name in all_curves:
        X.append(all_curves[name])
        Y.append(param_df.loc[name].values)
X = np.array(X)
Y = np.array(Y)
print(f'numb：{len(X)}')

scaler_X = StandardScaler().fit(X)
scaler_Y = StandardScaler().fit(Y)
X_scaled = scaler_X.transform(X)[..., np.newaxis]
Y_scaled = scaler_Y.transform(Y)

import joblib
joblib.dump(scaler_X, 'scaler_X.pkl')
joblib.dump(scaler_Y, 'scaler_Y.pkl')

print("ok")

X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, Y_scaled, test_size=0.2, random_state=42
)

In [ ]:
class CurveHyperModel(HyperModel):
    def build(self, hp):
        inp = Input(shape=(200, 1))

        
        num_cnn_layers = hp.Int('cnn_layers', 1, 3)
        x = inp
        for i in range(num_cnn_layers):
            filters = hp.Choice(f'filters_{i}', [32, 64, 128])
            kernel_size = hp.Choice(f'kernel_{i}', [3, 5])
            activation = hp.Choice(f'act_cnn_{i}', ['relu', 'tanh'])
            x = Conv1D(filters, kernel_size, activation=activation, padding='same')(x)

        
        pos_embed = tf.keras.layers.Embedding(input_dim=200, output_dim=x.shape[-1])(tf.range(0, 200))
        x = x + pos_embed

        
        attn = MultiHeadAttention(
            num_heads=hp.Choice('num_heads', [2, 4, 8]),
            key_dim=hp.Choice('key_dim', [16, 32])
        )(x, x)
        x = Add()([x, attn])
        x = LayerNormalization()(x)

        # FFN
        ffn_units = hp.Choice('ffn_units', list(range(96, 257, 16)))
        act_ffn = hp.Choice('act_ffn', ['relu', 'tanh'])
        ffn = Dense(ffn_units, activation=act_ffn)(x)
        ffn = Dense(x.shape[-1])(ffn)
        x = Add()([x, ffn])
        x = LayerNormalization()(x)

        # MLP Head
        x = GlobalAveragePooling1D()(x)
        x = Dropout(hp.Float('dropout1', 0.1, 0.5))(x)
        dense_units = hp.Choice('dense_units', list(range(96, 257, 16)))
        act_dense = hp.Choice('act_dense', ['relu', 'tanh'])
        x = Dense(dense_units, activation=act_dense)(x)
        x = Dropout(hp.Float('dropout2', 0.1, 0.5))(x)
        out = Dense(10)(x)

        model = Model(inputs=inp, outputs=out)
        lr = hp.Float('learning_rate', 1e-4, 5e-3, sampling='log')
        model.compile(optimizer=Adam(learning_rate=lr), loss='mse', metrics=['mae'])
        return model


tuner = Hyperband(
    CurveHyperModel(),
    objective='val_loss',
    max_epochs=100,
    factor=3,
    directory='tuner_dir',
    project_name='cnn_transformer_tune'
)

early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

tuner.search(X_train, Y_train,
             validation_split=0.1,
             epochs=100,
             batch_size=32,
             callbacks=[early_stop])


best_hp = tuner.get_best_hyperparameters(1)[0]
print("best：")
for key, value in best_hp.values.items():
    print(f"{key}: {value}")
    

best_model = tuner.get_best_models(1)[0]
best_model.save('best_model.h5')
print("save")

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model = tuner.get_best_models(1)[0]
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, Y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop]
)

In [ ]:
Y_pred = model.predict(X_test)
Y_pred_real = scaler_Y.inverse_transform(Y_pred)
Y_test_real = scaler_Y.inverse_transform(Y_test)

Y_pred_rounded = np.round(Y_pred_real)
Y_pred_rounded[:, 0:8] = np.clip(Y_pred_rounded[:, 0:8], 4, 13)      
Y_pred_rounded[:, 8] = np.clip(Y_pred_rounded[:, 8], 16, 112)         
Y_pred_rounded[:, 9] = np.clip(Y_pred_rounded[:, 9], 16, 112)       


In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

param_names = ['top','left','bottom','right','diag1','diag2','vert','hori','x','y']
for i, name in enumerate(param_names):
    r2 = r2_score(Y_test_real[:, i], Y_pred_rounded[:, i])
    mae = mean_absolute_error(Y_test_real[:, i], Y_pred_rounded[:, i])
    acc = np.mean(Y_test_real[:, i] == Y_pred_rounded[:, i]) * 100
    print(f"{name:<6} | R²: {r2:.3f} | MAE: {mae:.2f} | acc: {acc:.2f}%")


In [ ]:

plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Training & Validation Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()